In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np

Mounted at /content/drive


##Day 11: Dumbo Octopus

In [ ]:
filePath = '/content/drive/MyDrive/AdventOfCode/input11.txt'

filename = open(filePath, "r")
G = []
gr = filename.readline().strip()
while gr:
    gr = list(map(int,str(gr)))
    G.append(gr)
    gr = filename.readline().strip()

G = np.array(G)

In [ ]:
def one_step(G):
  G = np.pad(G, pad_width=1, mode='constant', constant_values=-99)
  row, col = len(G), len(G[0])
  flashed = np.ones([row,col])
  G = G+1
  
  not_finish = True
  while not_finish:
    not_finish = False
    for r in range(1,len(G)-1):
      for c in range(1,len(G[r])-1):
        if G[r][c] >= 10:
          G[r-1:r+2,c-1:c+2]+=1
          flashed[r][c] *= 0
          not_finish = True
    G = G*flashed
  total_flash = (row-2)*(col-2)-np.sum(flashed[1:row-1,1:col-1])
  return G[1:row-1,1:col-1], total_flash

In [ ]:
def multi_step(G, num):
  sum_flash = 0
  for i in range(num):
    G, flash = one_step(G)
    sum_flash += flash
  return sum_flash

print(multi_step(G,100))

1723.0


In [ ]:
def first_all_flash(G):
  count = 0
  all_flash = False
  while not all_flash:
    count += 1 
    G, flash = one_step(G)
    if flash == len(G)*len(G[0]):
      all_flash = True
  return count

print(first_all_flash(G))

327


##Day 12: Passage Pathing

In [ ]:
filePath = '/content/drive/MyDrive/AdventOfCode/input12.txt'
#filePath = '/content/drive/MyDrive/AdventOfCode/test.txt'

#data is a list of list containing each connections
data = [] 
with open(filePath) as file:
    for line in file:
        string = line.rstrip()
        data.append(list(string.split("-"))) 
print(data)
#cnct is a dictionary indicating which path can lead to where. 
#start can only be a key and will not be in value of any key. end will only be value, not key.
cnct = {}
for i in data:
    cnct[i[0]] = []
    cnct[i[1]] = []
for i in data: 
    if i[0]!='end' and i[1] != 'start':
      cnct[i[0]].append(i[1])
    if i[1]!='end' and i[0] != 'start':
      cnct[i[1]].append(i[0])
del cnct['end']

[['bm', 'XY'], ['ol', 'JS'], ['bm', 'im'], ['RD', 'ol'], ['bm', 'QI'], ['JS', 'ja'], ['im', 'gq'], ['end', 'im'], ['ja', 'ol'], ['JS', 'gq'], ['bm', 'AF'], ['RD', 'start'], ['RD', 'ja'], ['start', 'ol'], ['cj', 'bm'], ['start', 'JS'], ['AF', 'ol'], ['end', 'QI'], ['QI', 'gq'], ['ja', 'gq'], ['end', 'AF'], ['im', 'QI'], ['bm', 'gq'], ['ja', 'QI'], ['gq', 'RD']]


In [ ]:
k = cnct.keys()
print(k)
visited = dict.fromkeys(k, False)
print(visited)

for i in cnct:
  print(i, ': ', cnct[i])

dict_keys(['bm', 'XY', 'ol', 'JS', 'im', 'RD', 'QI', 'ja', 'gq', 'AF', 'start', 'cj'])
{'bm': False, 'XY': False, 'ol': False, 'JS': False, 'im': False, 'RD': False, 'QI': False, 'ja': False, 'gq': False, 'AF': False, 'start': False, 'cj': False}
bm :  ['XY', 'im', 'QI', 'AF', 'cj', 'gq']
XY :  ['bm']
ol :  ['JS', 'RD', 'ja', 'AF']
JS :  ['ol', 'ja', 'gq']
im :  ['bm', 'gq', 'end', 'QI']
RD :  ['ol', 'ja', 'gq']
QI :  ['bm', 'end', 'gq', 'im', 'ja']
ja :  ['JS', 'ol', 'RD', 'gq', 'QI']
gq :  ['im', 'JS', 'QI', 'ja', 'bm', 'RD']
AF :  ['bm', 'ol', 'end']
start :  ['RD', 'ol', 'JS']
cj :  ['bm']


In [ ]:
class Caves:

  def __init__(self, connect):
      self.nodes = connect.keys()
      self.size = len(connect)
      self.graph = connect
      self.allpath = 0

  def getAllPathUtil(self, u, d, visited, path):
      if u.islower() == True: 
        visited[u] = True
      path.append(u)
      if u == d:
        #print('2: ', path)
        self.allpath+=1
      else:
        for i in self.graph[u]:
          if visited[i] == False:
            self.getAllPathUtil(i, d, visited, path)
      path.pop()
      visited[u] = False

  def getAllPath(self, s, d):
     visited = dict.fromkeys(self.nodes, False)
     visited['end'] = False
     path = []
     self.getAllPathUtil(s, d, visited, path)
     return self.allpath
     #print(len(self.allpath))

In [ ]:
c = Caves(cnct)
cave1path = c.getAllPath('start', 'end')
print(cave1path)

3887


In [ ]:
class Caves2:
  #only counts the path that visit one cave twice.
  def __init__(self, connect):
      self.nodes = connect.keys()
      self.size = len(connect)
      self.graph = connect
      self.allpath = 0
      self.small = []

  def getAllPathUtil(self, u, d, visited, path, smol):
      if u.islower() == True: 
        visited[u] += 1
      path.append(u)
      if u == d:
        if path.count(smol) == 2:
          self.allpath+=1
      else:
        for i in self.graph[u]:
          if visited[i] < 1:
            self.getAllPathUtil(i, d, visited, path, smol)
      path.pop()
      visited[u] -= 1

  def getAllPath(self, s, d):
     for i in self.nodes:
       if i.islower() == True and i != 'start':
         self.small.append(i)

     for smol in self.small:
       visited = dict.fromkeys(self.nodes, 0)
       visited['end'] = 0
       visited[smol] = -1
       path = []
       
       self.getAllPathUtil(s, d, visited, path, smol)
     return self.allpath


In [ ]:
c2 = Caves2(cnct)
cave2path = c2.getAllPath('start', 'end')
total = cave1path+cave2path
print(cave1path, cave2path, total)

3887 100947 104834


##Day 13: Transparent Origami

In [ ]:
filePath = '/content/drive/MyDrive/AdventOfCode/input13.txt'

filename = open(filePath, "r")
G = []
gr = filename.readline().strip()
while gr:
    gr = list(map(int,gr.split(",")))
    G.append(gr)
    gr = filename.readline().strip()
#G = np.array(G)
print(G)

'''
fold along x=655
fold along y=447
fold along x=327
fold along y=223
fold along x=163
fold along y=111
fold along x=81
fold along y=55
fold along x=40
fold along y=27
fold along y=13
fold along y=6
'''

[[522, 329], [174, 466], [242, 714], [785, 686], [378, 596], [335, 353], [229, 44], [1160, 19], [5, 318], [489, 63], [484, 275], [818, 774], [214, 882], [1128, 519], [1143, 432], [627, 217], [669, 544], [306, 191], [878, 199], [579, 376], [649, 121], [1004, 191], [1017, 838], [519, 329], [62, 228], [249, 733], [1275, 108], [569, 173], [1297, 686], [23, 187], [5, 576], [823, 462], [730, 395], [1049, 309], [843, 35], [164, 364], [448, 373], [50, 372], [74, 723], [970, 539], [569, 33], [579, 605], [1126, 750], [801, 742], [525, 726], [293, 844], [334, 686], [21, 262], [514, 617], [1303, 541], [816, 136], [211, 284], [283, 833], [887, 119], [800, 823], [559, 131], [1077, 695], [579, 206], [764, 350], [492, 120], [1103, 803], [229, 850], [582, 785], [1124, 442], [676, 863], [897, 350], [45, 32], [126, 777], [1256, 621], [853, 862], [550, 771], [1299, 723], [328, 728], [907, 737], [296, 310], [110, 592], [780, 850], [822, 72], [1081, 44], [964, 649], [443, 817], [681, 8], [853, 638], [796, 6

'\nfold along x=655\nfold along y=447\nfold along x=327\nfold along y=223\nfold along x=163\nfold along y=111\nfold along x=81\nfold along y=55\nfold along x=40\nfold along y=27\nfold along y=13\nfold along y=6\n'

In [ ]:
def fold_along_x (G, x):
  for dot in G:
    if dot[0]>x:
      new_x = x-(dot[0]-x)
      dot[0] = new_x
  return G

def fold_along_y (G, y):
  for dot in G:
    if dot[1]>y:
      new_y = y-(dot[1]-y)
      dot[1] = new_y
  return G

fold1 = fold_along_x (G, 655)
#print(fold1)
#print(len(fold1))
nonRepeat = [i for n, i in enumerate(fold1) if i not in fold1[:n]]

print(len(nonRepeat))

[[522, 329], [174, 466], [242, 714], [525, 686], [378, 596], [335, 353], [229, 44], [150, 19], [5, 318], [489, 63], [484, 275], [492, 774], [214, 882], [182, 519], [167, 432], [627, 217], [641, 544], [306, 191], [432, 199], [579, 376], [649, 121], [306, 191], [293, 838], [519, 329], [62, 228], [249, 733], [35, 108], [569, 173], [13, 686], [23, 187], [5, 576], [487, 462], [580, 395], [261, 309], [467, 35], [164, 364], [448, 373], [50, 372], [74, 723], [340, 539], [569, 33], [579, 605], [184, 750], [509, 742], [525, 726], [293, 844], [334, 686], [21, 262], [514, 617], [7, 541], [494, 136], [211, 284], [283, 833], [423, 119], [510, 823], [559, 131], [233, 695], [579, 206], [546, 350], [492, 120], [207, 803], [229, 850], [582, 785], [186, 442], [634, 863], [413, 350], [45, 32], [126, 777], [54, 621], [457, 862], [550, 771], [11, 723], [328, 728], [403, 737], [296, 310], [110, 592], [530, 850], [488, 72], [229, 44], [346, 649], [443, 817], [629, 8], [457, 638], [514, 617], [162, 728], [432,

In [ ]:
def foldAll(G, X, Y):
  for x in X:
    G = fold_along_x (G, x)
  for y in Y:
    G = fold_along_y (G, y)
  
  nonRepeatG = [i for n, i in enumerate(G) if i not in G[:n]]
  return nonRepeatG


def visualize(G, X, Y):
  canvas = ['.'*X]*Y
  for dot in G:
    x = dot[0]
    y = dot[1]
    canvas[y] = canvas[y][:x] + '#' +canvas[y][x+1:]
  for line in canvas:
    print(line)

x = [655, 327, 163, 81, 40]
y = [447, 223, 111, 55, 27, 13, 6]

folded = foldAll(G, x, y)
print(len(folded))
visualize(folded, 40, 6)

99
####.###..####..##..#..#..##..#..#.#..#.
#....#..#....#.#..#.#.#..#..#.#..#.#..#.
###..#..#...#..#....##...#....####.#..#.
#....###...#...#.##.#.#..#....#..#.#..#.
#....#....#....#..#.#.#..#..#.#..#.#..#.
####.#....####..###.#..#..##..#..#..##..


##Day 14: Extended Polymerization

In [ ]:
filePath = '/content/drive/MyDrive/AdventOfCode/input14.txt'
#filePath = '/content/drive/MyDrive/AdventOfCode/test.txt'

data = [] 
with open(filePath) as file:
    starter = file.readline().rstrip()
    file.readline()
    for line in file:
        string = line.rstrip()
        data.append(list(string.split(" -> "))) 

print(starter)
print(data)

rules = {}
for i in data:
    rules[i[0]] = i[1]

print(rules)

KHSNHFKVVSVPSCVHBHNP
[['FV', 'H'], ['SB', 'P'], ['NV', 'S'], ['BS', 'K'], ['KB', 'V'], ['HB', 'H'], ['NB', 'N'], ['VB', 'P'], ['CN', 'C'], ['CF', 'N'], ['OF', 'P'], ['FO', 'K'], ['OC', 'F'], ['BN', 'V'], ['PO', 'O'], ['OS', 'B'], ['KH', 'N'], ['BB', 'C'], ['PV', 'K'], ['ON', 'K'], ['NF', 'H'], ['BV', 'K'], ['SN', 'N'], ['PB', 'S'], ['PK', 'F'], ['PF', 'S'], ['BP', 'K'], ['SP', 'K'], ['NN', 'K'], ['FP', 'N'], ['NK', 'N'], ['SF', 'P'], ['HS', 'C'], ['OH', 'C'], ['FS', 'H'], ['VH', 'N'], ['CO', 'P'], ['VP', 'H'], ['FF', 'N'], ['KP', 'B'], ['BH', 'B'], ['PP', 'F'], ['SS', 'P'], ['CV', 'S'], ['HO', 'P'], ['PN', 'K'], ['SO', 'O'], ['NO', 'O'], ['NH', 'V'], ['HH', 'F'], ['KK', 'C'], ['VO', 'B'], ['KS', 'B'], ['SV', 'O'], ['OP', 'S'], ['VK', 'H'], ['KF', 'O'], ['CP', 'H'], ['SH', 'H'], ['NC', 'S'], ['KC', 'O'], ['CK', 'H'], ['CH', 'B'], ['KO', 'O'], ['OV', 'P'], ['VF', 'V'], ['HN', 'P'], ['FH', 'P'], ['BC', 'V'], ['HV', 'N'], ['BO', 'V'], ['PH', 'P'], ['NP', 'F'], ['FN', 'F'], ['FK', 'P'], ['S

In [ ]:
def polymerization(starter, rules, rounds):
    for r in range(rounds):
      newstring = ''
      for i in range(len(starter)-1):
        key = starter[i]+starter[i+1]
        newstring = newstring + starter[i] + rules[key]
        i +=1 
      newstring = newstring + starter[-1]
      starter = newstring
    return starter

ten_session = polymerization(starter, rules, 10)

def count_elements(string1):
    dic = {}
    for i in string1:
        if i in dic:
            dic[i] +=1
        else:
            dic[i] = 1
    return dic
print(count_elements(ten_session))

{'K': 2040, 'V': 1614, 'B': 1762, 'P': 3258, 'F': 1636, 'N': 1615, 'H': 1531, 'C': 850, 'S': 2787, 'O': 2364}


In [ ]:
print(3258-850)

In [ ]:
# For second part, I decide to use a different approach than first part. Instead of keeping long string, 
# I will just keep a dictionary recording the occurance of each pair of letters. eg, NNCG will be NN:1, NC:1, CG:1.

def polymerization2(starter, rules, rounds):
    dic1 = turn_to_dict(starter)
    for r in range(rounds):
        dic2 = {}
        for k in dic1:
            count = dic1[k]
            s1 = k[0] + rules[k] 
            s2 = rules[k] + k[1]
            if s1 in dic2:
                dic2[s1] += count
            else:
                dic2[s1] = count
            
            if s2 in dic2:
                dic2[s2] += count
            else:
                dic2[s2] = count
        dic1 = dic2
    return dic1

def turn_to_dict(starter):
    dic1 = {}
    for i in range(len(starter)-1):
      key = starter[i]+starter[i+1]
      if key in dic1:
        dic1[key] += 1
      else:
        dic1[key] = 1
    return dic1

def count_element_in_dict(dicta, starter):
    count_dict = {}
    for key in dicta:
      if key[0] in count_dict:
        count_dict[key[0]] += dicta[key]
      else:
        count_dict[key[0]] = dicta[key]
    if starter[-1] in count_dict:
      count_dict[starter[-1]] += 1 
    else:
      count_dict[starter[-1]] = 1
    return count_dict

def get_result(starter, rules, rounds): 
    dictionary = polymerization2(starter, rules, rounds)
    elements = count_element_in_dict(dictionary, starter)
    list_element_count = []
    for e in elements:
      list_element_count.append(elements[e])
    list_element_count.sort()
    return list_element_count[-1] - list_element_count[0]


r = get_result(starter, rules, 40)
print(r)

2651311098752


##Day 15: Chiton

could use A* 
could also use viterbi... unsure if the problem allows to go up and left...

## Day 16: Packet Decoder

2408


In [ ]:
input = 'E0529D18025800ABCA6996534CB22E4C00FB48E233BAEC947A8AA010CE1249DB51A02CC7DB67EF33D4002AE6ACDC40101CF0449AE4D9E4C071802D400F84BD21CAF3C8F2C35295EF3E0A600848F77893360066C200F476841040401C88908A19B001FD35CCF0B40012992AC81E3B980553659366736653A931018027C87332011E2771FFC3CEEC0630A80126007B0152E2005280186004101060C03C0200DA66006B8018200538012C01F3300660401433801A6007380132DD993100A4DC01AB0803B1FE2343500042E24C338B33F5852C3E002749803B0422EC782004221A41A8CE600EC2F8F11FD0037196CF19A67AA926892D2C643675A0C013C00CC0401F82F1BA168803510E3942E969C389C40193CFD27C32E005F271CE4B95906C151003A7BD229300362D1802727056C00556769101921F200AC74015960E97EC3F2D03C2430046C0119A3E9A3F95FD3AFE40132CEC52F4017995D9993A90060729EFCA52D3168021223F2236600ECC874E10CC1F9802F3A71C00964EC46E6580402291FE59E0FCF2B4EC31C9C7A6860094B2C4D2E880592F1AD7782992D204A82C954EA5A52E8030064D02A6C1E4EA852FE83D49CB4AE4020CD80272D3B4AA552D3B4AA5B356F77BF1630056C0119FF16C5192901CEDFB77A200E9E65EAC01693C0BCA76FEBE73487CC64DEC804659274A00CDC401F8B51CE3F8803B05217C2E40041A72E2516A663F119AC72250A00F44A98893C453005E57415A00BCD5F1DD66F3448D2600AC66F005246500C9194039C01986B317CDB10890C94BF68E6DF950C0802B09496E8A3600BCB15CA44425279539B089EB7774DDA33642012DA6B1E15B005C0010C8C917A2B880391160944D30074401D845172180803D1AA3045F00042630C5B866200CC2A9A5091C43BBD964D7F5D8914B46F040'
bin = "{0:08b}".format(int(input, 16))
print(bin)

1110000001010010100111010001100000000010010110000000000010101011110010100110100110010110010100110100110010110010001011100100110000000000111110110100100011100010001100111011101011101100100101000111101010001010101000000001000011001110000100100100100111011011010100011010000000101100110001111101101101100111111011110011001111010100000000000010101011100110101011001101110001000000000100000001110011110000010001001001101011100100110110011110010011000000011100011000000000101101010000000000111110000100101111010010000111001010111100111100100011110010110000110101001010010101111011110011111000001010011000000000100001001000111101110111100010010011001101100000000001100110110000100000000011110100011101101000010000010000010000000100000000011100100010001001000010001010000110011011000000000001111111010011010111001100111100001011010000000000000100101001100100101010110010000001111000111011100110000000010101010011011001011001001101100110011100110110011001010011101010010011000100000001100000000010011111001000